In [5]:
import pickle,h5py,os,keras,json, os, argparse, imp
import tensorflow as tf
import numpy as np

In [6]:
%load_ext autoreload
%autoreload 2
import utils as ut

In [7]:
# intializations 
path_to_data = # Data path of the created dictionary
filename = 'All_activity_instances_data.h5'



In [8]:
pickle_out = open('Indices_Train_Val.pkl',"rb")
temp = pickle.load(pickle_out) # indices of validation and test set
pickle_out.close()
indices_array_train = temp[0]
indices_array_Val = temp[1]
new_labels_train = temp[2]
new_labels_Val = temp[3]

Making data

In [9]:
data_file = h5py.File(os.path.join(path_to_data,filename), 'r') 
X_data = data_file['X']

In [10]:
X_train = np.zeros((indices_array_train.shape[0],X_data.shape[1],X_data.shape[2]))
X_Val = np.zeros((indices_array_Val.shape[0],X_data.shape[1],X_data.shape[2]))

for i in range(indices_array_train.shape[0]):
    X_train[i,:,:] = X_data[indices_array_train[i],:,:]
for i in range(indices_array_Val.shape[0]):
    X_Val[i,:,:] = X_data[indices_array_Val[i],:,:]


In [11]:
y_train = ut.convert_one_hot(new_labels_train)
y_Val = ut.convert_one_hot(new_labels_Val)

# Source Network

In [13]:
modelPath = "./models/"
modelName = "Stacked_LSTM_classifier"

In [50]:
def get_model(inp_shp,num_classes):
    inputs = keras.Input(shape=inp_shp, name="digits")
    x1 = keras.layers.LSTM(50,return_sequences = False, activation="relu", name="LSTM_1" )(inputs)
#     x = keras.layers.LSTM(64,return_sequences = False, activation="relu", name="LSTM_2" )(x)
#     x = keras.layers.Dense(32, activation="relu", name="dense_1")(x)
    x2 = keras.layers.Dense(num_classes, name="dense_2")(x1)
    outputs = keras.layers.Activation('softmax',name="activation")(x2)
    functional_model = keras.Model(inputs=inputs, outputs=outputs, name="Stacked_LSTM")
    return functional_model

def get_model_2(inp_shp,num_classes):
    model = model_check(inp_shp)
    inputs = keras.Input(shape=inp_shp, name="digits")
    features = model(inputs)
    classifier = keras.layers.Dense(num_classes,use_bias = False)(features)
    outputs = keras.layers.Activation('softmax',name="activation")(classifier)
    functional_model = keras.Model(inputs=inputs, outputs=outputs, name="Stacked_LSTM")
    return functional_model


def model_check(input_shape):
#     inp = Input(input_shape)
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(128,dropout = 0.2,return_sequences = True,input_shape = input_shape))
    model.add(keras.layers.LSTM(100,dropout = 0.2,return_sequences= False,input_shape = input_shape))
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(40))
    model.add(keras.layers.Dropout(0.2))
#     model.add(keras.layers.Activation('softmax'))
    return model

In [51]:
source_model = get_model_2((250,117),26)
# source_model = model_check((250,117))
source_model.summary()

Model: "Stacked_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          (None, 250, 117)          0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 40)                226616    
_________________________________________________________________
dense_9 (Dense)              (None, 26)                1040      
_________________________________________________________________
activation (Activation)      (None, 26)                0         
Total params: 227,656
Trainable params: 227,656
Non-trainable params: 0
_________________________________________________________________


In [52]:
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

print('(+) Saving model: '+modelName)
json_string = source_model.to_json()
open(os.path.join(modelPath, modelName+'.json'), 'w').write(json_string)
ad =keras.optimizers.Adam(lr = 0.01)
source_model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=ad, metrics=['acc'])
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=os.path.join(modelPath,modelName+'2.wgt'), 
                                verbose=1, 
                                save_best_only=True,monitor = 'val_acc')

(+) Saving model: Stacked_LSTM_classifier


In [54]:
# source_model.load_weights(os.path.join(modelPath, modelName+'1.wgt'))

In [55]:
fitlog = source_model.fit(X_train,y_train,batch_size = 64,epochs=10, verbose=1, 
                    callbacks=[checkpointer],
                    shuffle = "batch",validation_data = (X_Val,y_Val))
source_model.save_weights(filepath=os.path.join(modelPath,modelName+'1lastepoch.wgt'), overwrite=True)

Train on 7800 samples, validate on 520 samples
Epoch 1/10
7800/7800 [==============================] - 46s 6ms/step - loss: 1.5488 - acc: 0.4917 - val_loss: 0.7451 - val_acc: 0.7462

Epoch 00001: val_acc improved from -inf to 0.74615, saving model to ./models/Stacked_LSTM_classifier2.wgt
Epoch 2/10
7800/7800 [==============================] - 46s 6ms/step - loss: 0.9552 - acc: 0.6660 - val_loss: 0.4739 - val_acc: 0.8096

Epoch 00002: val_acc improved from 0.74615 to 0.80962, saving model to ./models/Stacked_LSTM_classifier2.wgt
Epoch 3/10
7800/7800 [==============================] - 46s 6ms/step - loss: 0.7095 - acc: 0.7646 - val_loss: 0.3452 - val_acc: 0.8942

Epoch 00003: val_acc improved from 0.80962 to 0.89423, saving model to ./models/Stacked_LSTM_classifier2.wgt
Epoch 4/10
7800/7800 [==============================] - 45s 6ms/step - loss: 0.4985 - acc: 0.8424 - val_loss: 0.2048 - val_acc: 0.9269

Epoch 00004: val_acc improved from 0.89423 to 0.92692, saving model to ./models/Stack

In [ ]:
pretrained = keras.Model(
    source_model.inputs, source.layers[-1].input, name="pretrained_model"
)

In [83]:
source_model.layers[-2].weights[0].shape

TensorShape([Dimension(117), Dimension(200)])

In [40]:
# source_model.predict(X_Val)
w_emp = source_model.layers[-2].weights

In [57]:
source_model.layers[-2].get_weights()[0].shape

(40, 26)

In [29]:
source_model.layers[-1].predict(X_Val).shape

AttributeError: 'Activation' object has no attribute 'predict'